#Data Cleaning On 311 Service Requests from 2010 to present

Note : Recommended to Run on google colab(https://colab.research.google.com/) for reproducibility

###Installing packages

In [ ]:
!pip install pyspark

In [ ]:
!pip install openclean[full]

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install humanfriendly

In [ ]:
!pip install -U -q PyDrive

###Loading Libraries

In [ ]:
# Loading Libraries
import pandas as pd
import numpy as np
from openclean.profiling.dataset import dataset_profile
from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint
from openclean.operator.map.violations import fd_violations
from openclean.operator.collector.count import distinct
from openclean.function.eval.logic import And
from openclean.function.value.phonetic import Soundex, soundex
from openclean.pipeline import stream
from openclean.data.load import dataset
from openclean.cluster.key import KeyCollision
from openclean_geo.address.usstreet import USStreetNameKey
from openclean_geo.address.usstreet import StandardizeUSStreetName
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,col,when,udf,asc,regexp_replace,desc,countDistinct
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.ml.feature import *
from openclean.profiling.column import DefaultColumnProfiler
from openclean.operator.map.violations import fd_violations
from openclean.operator.collector.count import distinct
import gzip
import humanfriendly
import os
import csv
from openclean.data.source.socrata import Socrata
from openclean.cluster.knn import knn_clusters, knn_collision_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan
from openclean.function.value.null import is_empty
from openclean.operator.transform.update import update
from openclean.function.eval.base import Col, Eval
from openclean.function.eval.logic import And
from openclean.function.value.phonetic import Soundex, soundex
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

### Loading dataset 

Since Our dataset is of size 15 GB with 27.3 Million rows and 41 Column , We are having two approaches below to load dataset.

1. Sample Dataset - 750 MB and 1.3 Million rows ( Use this for Reproducibility testing and faster analysis)
2. Socrata Api    - Complete Dataset ( Uncomment code  only if running on spark cluster)

#### Downloading reference data files from Google Drive

Accept the google sdk T&C from any google account and paste the link and press enter.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fileDownloaded = drive.CreateFile({'id':'1OMPb63gnxV3JRWNLeCWDbe1uB8l4_bBY'})
fileDownloaded.GetContentFile('nyc_zip_borough.csv')

In [ ]:
fileDownloaded = drive.CreateFile({'id':'1OMwBZPIM_Wy0n3QZzgA904TnX5Vv6Rq-'})
fileDownloaded.GetContentFile('Sampled_Data.csv')

#### Sample Dataset Loading

######Creating Spark Dataframe 

In [ ]:
spark = SparkSession.builder.appName("Project").getOrCreate()
sc = spark.sparkContext

In [ ]:
sc

<SparkContext master=local[*] appName=Project>

In [ ]:
df_data = spark.read.format("csv") \
.option("header", "true") \
.option("infer_schema","true")\
.option("first_row_is_header","true")\
.load("Sampled_Data.csv")

df_data.show()

+--------+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------------------+-----------------+---------------+---------------------+---------------------+------------+-------------------+--------+-------------+-------+--------------------+----------------------+------------------------------+--------------------+------------+-------------+--------------------------+--------------------------+----------------------+------------------+-------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|     _c0|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|       Location Type|Incident Zip|    Incident Address|        Street Name|   Cross Street

In [ ]:
df_311= df_data.select('*')
df_311.show()

+--------+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------------------+-----------------+---------------+---------------------+---------------------+------------+-------------------+--------+-------------+-------+--------------------+----------------------+------------------------------+--------------------+------------+-------------+--------------------------+--------------------------+----------------------+------------------+-------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|     _c0|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|       Location Type|Incident Zip|    Incident Address|        Street Name|   Cross Street

######Creating Data Stream 

In [ ]:
ds = stream("Sampled_Data.csv")

#### Complete Dataset Loading

Download file using Socrata API

In [ ]:
# dataset = Socrata().dataset('erm2-nwe9')
# datafile = './erm2-nwe9.tsv.gz'
# # Download file only if it does not exist already.
# if not os.path.isfile(datafile):
#     with gzip.open(datafile, 'wb') as f:
#       print('Downloading ...\n')
#       dataset.write(f)
# fsize = humanfriendly.format_size(os.stat(datafile).st_size)
# print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

##### Creating Spark Dataframe

In [ ]:
# spark = SparkSession.builder.appName("Project").getOrCreate()
# sc = spark.sparkContext

In [ ]:
# sc

In [ ]:
# df_data = spark.read.format("csv") \
# .option("header", "true") \
# .option("infer_schema","true")\
# .option("first_row_is_header","true")\
# .load(datafile)

In [ ]:
# df_311= df_data.select('*')
# df_311.show()

##### Creating Data Stream

In [ ]:
# ds = stream(datafile)

### Initial Profiling

In [ ]:
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname PLACE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,total,empty,distinct,uniqueness,entropy
,1353465,0,1353465,1.000000,20.368226
Unique Key,1353465,0,1353465,1.000000,20.368226
Created Date,1353465,0,1142338,0.844010,19.440945
Closed Date,1353465,37626,939005,0.713617,18.714398
Agency,1353465,0,28,0.000021,2.949971
Agency Name,1353465,0,680,0.000502,3.336534
Complaint Type,1353465,0,316,0.000233,5.921032
Descriptor,1353465,21326,1484,0.001114,7.334426
Location Type,1353465,320891,171,0.000166,3.129963
Incident Zip,1353465,71169,823,0.000642,8.012380


In [ ]:
print(profiles.types())

                                   date   float      int     str
                                      0       0  1353465       0
Unique Key                            0       0  1353465       0
Created Date                    1142338       0        0       0
Closed Date                      939005       0        0       0
Agency                                0       0        0      28
Agency Name                           0       0        0     680
Complaint Type                        0       0        0     316
Descriptor                            0       0        0    1484
Location Type                         0       0        0     171
Incident Zip                          0     334      441      48
Incident Address                     60       0       23  429512
Street Name                           1       0       48   16039
Cross Street 1                        2       0        3   21357
Cross Street 2                        0       0        4   18131
Intersection Street 1    

In [ ]:
df_data.schema.names 

['_c0',
 'Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location']

###Initial Cleaning and Dimensionality Reduction



###### Drop Duplicate rows

In [ ]:
''' droping duplicate rows '''
df_311.dropDuplicates().show()

+--------+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-----------------+---------------+---------------+---------------------+---------------------+------------+-----+--------+-------------+-------+--------------------+----------------------+------------------------------+-----------------+------------+-------+--------------------------+--------------------------+----------------------+------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+-----------------+------------------+--------------------+
|     _c0|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|       Location Type|Incident Zip|    Incident Address|      Street Name| Cross Street 1| Cross Street 2|Intersection S

######Making Null's Consistent across dataset

Data contains Unknown value not in in the form of Nan,nulls but also with values 'Unspecified' and 'UNKOWN' .
We are replacing this values with nulls to make sure it's consistent across dataset.

In [ ]:
''' Replace Unspecified and UNKOWN value with nulls'''
def replace(column, value):
    return when(column != value, column).otherwise(lit(None))

df_311 = df_311.withColumn("Park Facility Name", replace(col("Park Facility Name"), "Unspecified"))
df_311 = df_311.withColumn("Open Data Channel Type", replace(col("Open Data Channel Type"), "UNKNOWN"))

df_311.show()

+--------+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------------------+-----------------+---------------+---------------------+---------------------+------------+-------------------+--------+-------------+-------+--------------------+----------------------+------------------------------+--------------------+------------+-------------+--------------------------+--------------------------+----------------------+------------------+-------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|     _c0|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|       Location Type|Incident Zip|    Incident Address|        Street Name|   Cross Street

######Dimensionality Reduction

As seen in profiling there are many column which has null or unspecifed values and hence will be of no use for future analysis . Hence we are removing the columns where more than 50% of data is null.

In [ ]:
''' Drop columns with more than 50% of rows being null'''

import pyspark.sql.functions as F
def drop_null_columns(df):
    """
    This function drops all columns which contain null values.
    :param df: A PySpark DataFrame
    """
    total_rows = df.count()
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > (0.5*total_rows)]
    print('to_drop:{}'.format(to_drop))
    df = df.drop(*to_drop)
    return df

df_311= drop_null_columns(df_311)

to_drop:['Intersection Street 1', 'Intersection Street 2', 'Landmark', 'Facility Type', 'Due Date', 'Park Facility Name', 'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment']


There are some duplicate columns like 

*   Location - Combination of Longitude and Latitude Columns
*   Park Borough - Same as Borough Column 

So we are removing this column as part of dimensionality reduction.

In [ ]:
''' Drop duplicate and unnecessary Columns'''

df_311=df_311.drop('Location')
df_311=df_311.drop('Park Borough')
df_311 = df_311.drop("_c0")

###Creating Spark SQL View

After the Dimensionality Reduction we now require to clean data according to respective Column and spark sql could be great use to analyze data.

In [ ]:
df_311.createOrReplaceTempView("df_311")


###Cleaning DATE Columns

In our dataset we have 3 Date Coloumns 
1.   Created Date
2.   Closed Date
3.   Resolution Action Updated Date


i. We remove all entries where the closed date, resolution action action date both are null and status is either closed or null

In [ ]:
df_311 = spark.sql("SELECT * FROM df_311 where `Closed Date` is not null and (status is not null or status!='Closed') and `Resolution Action Updated Date` is not null")

ii. We fill the missing closed date values based on the resolution action update date and vice versa

In [ ]:
df_311=df_311.withColumn("Closed Date", when((col("Closed Date").isNull()) & (col("Status")=='Closed') &(col("Resolution Action Updated Date").isNotNull()),col("Resolution Action Updated Date")).otherwise(col("Closed Date")))
df_311=df_311.withColumn("Resolution Action Updated Date", when((col("Resolution Action Updated Date").isNull()) & (col("Closed Date").isNotNull()),col("Closed Date")).otherwise(col("Resolution Action Updated Date")))

iii. Remove all entries where closed date < created date and created date > current date

In [ ]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df_311.createOrReplaceTempView("df_311_closed_date")
df_311=spark.sql("select * from df_311_closed_date where to_date(`closed date`,'MM/dd/yyyy')>=to_date(`created date`,'MM/dd/yyyy')")
df_311=spark.sql("select * from df_311_closed_date where to_date(`created date`,'MM/dd/yyyy')<=now() or to_date(`closed date`,'MM/dd/yyyy')<=now()")

### Cleaning AGENCY and AGENCY NAME Columns

In the Below outputs we find that **"MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT"** in agency should have been **"OSE"** as short form and **"Mayor's Special Office of Enforcement"** as complete Agency Name.



In [ ]:
df_311.select("Agency").distinct().show()

+--------------------+
|              Agency|
+--------------------+
|               NYCEM|
|MAYORâS OFFICE ...|
|                 DOT|
|                 HPD|
|                 DCA|
|                 DPR|
|                 TLC|
|                 EDC|
|                 DOF|
|               DORIS|
|               3-1-1|
|                NYPD|
|                 DOB|
|                DFTA|
|                 DEP|
|                 DOE|
|               DOHMH|
|                DSNY|
|                 DHS|
|               DOITT|
+--------------------+



In [ ]:
df_311.select("Agency Name").distinct().show()

+--------------------+
|         Agency Name|
+--------------------+
|      Exemption Unit|
|School - PS 200 B...|
|Department of Env...|
|        Manhattan 02|
|School - Metropol...|
|School - PS 199 J...|
|   Brooklyn South 10|
|School - PS 81 Th...|
|BCC - Brooklyn North|
|School - JHS 8 Ri...|
|       A - Manhattan|
|School - Fort Gre...|
|School - PS 116 E...|
|School - William ...|
|School - School f...|
|Economic Developm...|
|School - Stuyvesa...|
|Taxi and Limousin...|
|School - PS 50 Vi...|
|School - PS 105 S...|
+--------------------+
only showing top 20 rows



Befor Cleaning Agency and Agency Name Visualization

In [ ]:
from pyspark.sql.window import Window
agency_counts = df_311.groupBy("Agency").count().orderBy("count",ascending=False)
agency_counts.show()

+--------------------+------+
|              Agency| count|
+--------------------+------+
|                NYPD|382606|
|                 HPD|331687|
|                 DOT|171203|
|                DSNY|107311|
|                 DEP| 99371|
|                 DOB| 67736|
|                 DPR| 47069|
|               DOHMH| 25847|
|                 DOF| 25172|
|                 TLC| 12705|
|                 DCA| 12573|
|                 DHS| 11882|
|                DFTA|  4779|
|MAYORâS OFFICE ...|  3460|
|                 EDC|  1207|
|                 DOE|  1029|
|               NYCEM|   423|
|               DOITT|   367|
|               3-1-1|   272|
|               DORIS|     3|
+--------------------+------+



In [ ]:
agency_percentage = agency_counts.withColumn("percent", f.col('count')*100/f.sum('count').over(Window.partitionBy())) 
agency_percentage.orderBy('percent', ascending=False).show()

NameError: ignored

Below code makes the required changes

In [ ]:
df_311 = df_311.withColumn("Agency", when(df_311.Agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT',"OSE").otherwise(df_311.Agency))
df_311 = df_311.withColumn("Agency Name",when(df_311["Agency Name"][0:6] == "Mayorâ", "Mayor's Special Office of Enforcement").otherwise(df_311["Agency Name"]))

These columns are now having consitent and clean data

In [ ]:
df_311.select("Agency", "Agency Name").filter(df_311["Agency"] == "OSE").show(20,False)

Below we are Trying to check the Functional dependancy between Agency and Agency Name i.e, a single agency can have many Agency Name but Agency name only corresponds to single Agency. 
We do not find any conflicts so no more cleaning is required.

In [ ]:
df_311.groupBy("Agency Name").agg(countDistinct("Agency")).show(2000, False)

### Cleaning COMPLAINT TYPE Column

In [ ]:
df_311.groupBy("Complaint Type").count().sort(desc("count")).show(2000, False)

To make analysis easier we tried to group some of the above Complaint types in a single group for example in above output we have 

1.   Noise                                    
2.   Noise - Commercial                       
3.   Noise - Helicopter                       
4.   Noise - House of Worship                 
5.   Noise - Park                             
6.   Noise - Residential                      
7.   Noise - Street/Sidewalk                  
8.   Noise - Vehicle                          
9.   Noise Survey 

This all could be grouped under a single Complaint Type Noise

In [ ]:
df_311=df_311.withColumn("Complaint Type", regexp_replace(df_311["Complaint Type"],"Ferry.*","Ferry Complaint"))
df_311=df_311.withColumn("Complaint Type", regexp_replace(df_311["Complaint Type"],"Highway.*","Highway Complaint"))
df_311=df_311.withColumn("Complaint Type", regexp_replace(df_311["Complaint Type"],"Noise.*","Noise Complaint"))
df_311=df_311.withColumn("Complaint Type", regexp_replace(df_311["Complaint Type"],"Taxi.*","Taxi Complaint"))

We also find that there are some duplicate complaints like 
1.  Derelict Vehicle (10370) and Derelict Vehicles (13706)
2.  Animal-Abuse (875) and Animal Abuse (2740)
3.  PAINT - PLASTER (17827) and PAINT/PLASTER (20353)


We are using KNN cluster algorithm to group them and using the suggested word to replace and standardize it on whole dataset.



In [ ]:
complaint_type_dict= {}
cluster_dict ={}
def perform_knn_cluster(ds,column, using_collision=True, minsize=2, t=0.75):
    values = ds.select(column).distinct()
    clusters = knn_clusters(values=values, sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(t)), minsize=minsize) \
        if using_collision else knn_collision_clusters(values=values, sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(t)), minsize=minsize)
    print('{} clusters of size {} or greater'.format(len(clusters), minsize))
    clusters.sort(key=lambda c: len(c), reverse=True)
    for i, cluster in enumerate(clusters):
        print_cluster(i + 1, cluster)
        
def print_cluster(cnumber, cluster):
    print('Cluster {} (of size {})\n'.format(cnumber, len(cluster)))
    for val, count in cluster.items():
        print('{} ({})'.format(val, count))
        complaint_type_dict[val] = cnumber
    print('\nSuggested value: {}\n\n'.format(cluster.suggestion()))
    cluster_dict[cnumber] = cluster.suggestion()

perform_knn_cluster(ds,"Complaint Type")

In [ ]:
complaint_type_dict

In [ ]:
cluster_dict

UDF to update Complaint Type

In [ ]:
@udf
def update_complaint_type(complaint):
    if complaint in complaint_type_dict:
      return cluster_dict[complaint_type_dict[complaint]]
    return complaint

df_311= df_311.withColumnRenamed("Complaint Type","Complaint_Type")
df_311= df_311.withColumn("Complaint_Type", update_complaint_type(df_311.Complaint_Type))

Result

In [ ]:
df_311.groupBy("Complaint_Type").count().sort(desc("count")).show(2000, False)

### Cleaning INCIDENT ZIP Column

Remove all null rows

In [ ]:
df_311.na.drop(subset=["Incident Zip"]).show()

Having a look at values of Zip we find many values to be either decimal, alphanumeric or inconsistent length.

In [ ]:
df_311=df_311.withColumnRenamed("Incident Zip","Incident_Zip")
df_311.select("Incident_Zip").distinct().show(50)

Standardizing zip codes to length = 5 and only replacing values if numeric.

In [ ]:
@udf
def fill(zip_code):
    ''' Function takin zip code as input
        checks if zip code is numeric and has 5 digits 
        returns formated zip code or "" 
    '''
    zip_code = str(zip_code).zfill(5)
    if not zip_code.isnumeric() :
      return " "
    return zip_code[:5]

df_311= df_311.withColumn("Incident_Zip", fill(df_311.Incident_Zip))

Removing nulls and empty (" ") zip codes from dataset and since new york zip code ranges from 10001 - 11697 remove any zip code not in this range.

In [ ]:
df_311.createOrReplaceTempView("df_311_incident_zip")
df_311= spark.sql('Select * FROM df_311_incident_zip  WHERE Incident_Zip IS NOT NULL AND Incident_Zip <> " " AND CAST(Incident_Zip AS int)> 10000 AND CAST(Incident_Zip as int) <= 11697 ')     

Result

In [ ]:
df_311.select('Incident_Zip').distinct().orderBy(col('Incident_Zip').desc()).show()

### Cleaning CITY Column

We find that the CASE of city names is inconsistent as seen in below output we have 


1.   Corona and CORONA
2.   Bayside and BAYSIDE



In [ ]:
df_311.select("City").distinct().show()

We are normalizing the case and making sure city does not has numeric values

In [ ]:
@udf
def standardize(city_name):
  ''' takes city names as input and output in Init caps form if valid city name '''
  standard_city_name = str(city_name).strip().capitalize()
  if standard_city_name.isnumeric():
    return None
  return standard_city_name

df_311= df_311.withColumn("City", standardize(df_311.City))

Result

In [ ]:
df_311.select("City").distinct().show()

As a part of initial cleaning of some mispelled city names like Newarkk,BRKLYN.
We started with intial startegy of soundex but to our surprise some city names like newark has sam soundex of New York and this would make data rigged.

In [ ]:
newyork = ds\
    .select('City')\
    .update('City', str.upper)\
    .filter(And(Eval('City', Soundex()) == soundex('NEW YORK'), Col('City') != 'NEW YORK'))\
    .distinct()


As you see Newarkk soundex ranks highest for 'New York'

In [ ]:
print('RANK\tCOUNT\tNAME')
for i, entry in enumerate(newyork.most_common()):
    key, count = entry
    print('{}.\t{}\t{}'.format(i + 1, count, key))

Using Functional Dependency b/w City and Zip code to clean city column

Running the code below we find that a single zip code actually has mutiple city values or missing values which is incorrect.

In [ ]:
df_311.createOrReplaceTempView("df_311_city")
temp = spark.sql('Select Incident_Zip,CITY,Count(*) FROM df_311_city GROUP BY Incident_Zip,City Order by 1 ASC,3 DESC')
temp.show()

A simple zip code example with multiple values

In [ ]:
temp = spark.sql('Select Incident_Zip,City,COUNT(*) FROM df_311_city  WHERE Incident_Zip="11368" Group by Incident_Zip,City Order by 3 DESC ') 
temp.distinct().show()

Since we did not find any open or free dataset which could be used as a lookup for mapping city with zip code, We are trying to make use of our own strategy and assumption which you will also find mostcommon across our other cleaning techniques. 

As you can see above the frequency of Corona is higher than any other city for zip code 11368 and which is actually the correct mapping. Our assumption is that most of the data is correct and correct values are higher than incorrect values. So our strategy finds out the highest occuring values and replaces all the invalid or low occuring values with highest occuring values.


In [ ]:
''' creating ranks based on counts of each city for particular zip code '''
ranked_by_zip_city= spark.sql(" Select Incident_Zip, City ,COUNT(*),ROW_NUMBER() OVER(PARTITION BY Incident_Zip ORDER BY COUNT(*) DESC,Incident_Zip DESC) as rnk FROM df_311_city WHERE Incident_Zip IS NOT NULL GROUP BY Incident_Zip, City ORDER BY 1 ASC,3 DESC ") 
ranked_by_zip_city.show()

Getting the top ranked(most frequent) city name for each zip code and creating a dictionary with key as zip and value as most frequent city.  

In [ ]:
ranked_by_zip_city.createOrReplaceTempView("ranked_by_zip_city")
top_ranked = spark.sql(" SELECT Incident_Zip,City FROM ranked_by_zip_city WHERE rnk =1 ")

data_itr = top_ranked.rdd.toLocalIterator()
zip_dicti = {}
for row in data_itr:
    zip_dicti[row["Incident_Zip"]] = row["City"]

Updating the spark dataframe using the dictionary.

In [ ]:
@udf

def update_city(zip_code):
  ''' takes input as zip code and outputs the city the zip code refers to '''
  if zip_code in zip_dicti:
      return zip_dicti[zip_code]
  return " " 

df_311 =df_311.withColumn("City", update_city(df_311.Incident_Zip))
df_311.show()

Result

In [ ]:
df_311.createOrReplaceTempView('df_311_zip')
temp = spark.sql('Select Incident_Zip,City FROM df_311_zip  WHERE Incident_Zip="11368"') 
temp.distinct().show()

In [ ]:
df_311.select("City").distinct().orderBy("City").show(100)

### Cleaning Borough Column

Data is preceeded with numeric and followed by Borough

In [ ]:
df_311.select("Borough").distinct().show()

We also find some unspecified and null values which can be filled with functional dependancy b/w zip and borough.

In [ ]:
temp = spark.sql('Select Incident_Zip,Borough FROM df_311_zip  WHERE Incident_Zip="10452"') 
temp.distinct().show()

In order to clean the numeric and standardize the data we are using a lookup file which has maping between zip and borough.

In [ ]:
#using nyc_zip_borough.csv loading master data for lookup

boro_df = pd.read_csv("nyc_zip_borough.csv")

boro_dict ={}
for _, row in boro_df.iterrows():
    boro_dict[str(row['zip'])] = str(row['borough'])

print(boro_dict)

UDF (User Defined function) to update borough based on above dictionary

In [ ]:
@udf
def update_boro(zip_code):
  '''takes zip_code as input and outputs appropriate Borough'''
  if zip_code in boro_dict:
      return boro_dict[zip_code]
  return " " 
df_311 =df_311.withColumn("Borough", update_boro(df_311.Incident_Zip))
df_311.select("Incident_Zip","Borough").show()

Result

In [ ]:
df_311.createOrReplaceTempView("df_311_boro")
temp = spark.sql('Select Incident_Zip,Borough FROM df_311_boro WHERE Incident_Zip="10452"') 
temp.distinct().show()

We find Borough with "" because our look data only had "Queens","Brooklyn","Staten Island","Manhattan","Bronx" as bourgh but our dataset also has king county and other boroughs so we are going to filter them out.

In [ ]:
df_311.select('Borough').distinct().show()

In [ ]:
li =["Queens","Brooklyn","Staten Island","Manhattan","Bronx"]
df_311=df_311.filter(df_311.Borough.isin(li))
df_311.select('Borough').distinct().show()

###Cleaning INCIDENT_ADRESS , STREET ADDRESS, CROSS STREET 1, CROSS STREET 2

#### Incident Address

We are using Key Collison Clustering with default fingerprint key generator for all address types

In [ ]:
incident_address = ds.update('Incident Address', str.upper).distinct('Incident Address')
clusters = KeyCollision(func=Fingerprint()).clusters(incident_address)

Based on the clusters created we are also generating 2  additonal dictionaries i.e, incident_address_dict and cluster_dict

In [ ]:
incident_address_dict = {}
cluster_dict = {}
def print_k_clusters(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    # val_count = sum([len(c) for c in clusters])
    # print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')
          temp_dict[key] = int(cnt)
          cluster_dict[key] = i

        incident_address_dict[i] = temp_dict 
print_k_clusters(clusters)   

Incident_address_dict contains a mapping of structure { cluster_number : {street : number of time it got repeated }}

In [ ]:
incident_address_dict 

Cluster dict is a simple dictionary which maps each street to repsective cluster.

In [ ]:
cluster_dict

Using the same most frequent strategy we are creating a dictionary from incident_address_dict with key as cluster number and value as most frequent street for that cluster.

In [ ]:
final_addr = {}
for key,value in incident_address_dict.items():
    final_addr[key] = max(value, key=value.get)

print(final_addr)

UDF to update Incident address 

In [ ]:
@udf
def update_incident_address(incident_address):
    if incident_address in cluster_dict:
      return final_addr[cluster_dict[incident_address]]
    return " "

df_311= df_311.withColumnRenamed("Incident Address","Incident_Address")
df_311= df_311.withColumn("Incident_Address", update_incident_address(df_311.Incident_Address))



Positive Case Result

In [ ]:
df_311.createOrReplaceTempView("df_data_address")
temp= spark.sql('Select * FROM df_data_address  WHERE Incident_Address =  "WEST 27 STREET" ')  
temp.show()

Negative Case Result

In [ ]:
temp= spark.sql('Select * FROM df_data_address  WHERE Incident_Address =  "WEST   27 STREET" ')  
temp.show()

There can be a possibilty of street name having multple spaces adn being most frequent so below is UDF to remove multiple spaces with a single space.

In [ ]:
@udf

def replace_with_single_spaces(addr):
    return " ".join(addr.split())

In [ ]:
df_311= df_311.withColumn("Incident_Address", replace_with_single_spaces(df_311.Incident_Address))

####Street Name

Similar to Incident Adress

In [ ]:
street_names = ds.update('Street Name', str.upper).distinct('Street Name')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(street_names)

In [ ]:
street_name_dict = {}
cluster_dict = {}
def print_k_clusters_and_integrate_sn(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')

          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        street_name_dict[i] = temp_dict

print_k_clusters_and_integrate_sn(clusters)

In [ ]:
final_street = {}
for key,value in street_name_dict.items():
    final_street[key] = max(value, key=value.get)

In [ ]:
@udf
def update_street_name(street_name):
    if street_name in cluster_dict:
      return final_street[cluster_dict[street_name]]
    return " "
df_311= df_311.withColumnRenamed("Street Name","Street_Name")
df_311= df_311.withColumn("Street_Name", update_incident_address(df_311.Street_Name))
df_311= df_311.withColumn("Street_Name", replace_with_single_spaces(df_311.Street_Name))

####Cross Street 1

In [ ]:
cross_street_1 = ds.update('Cross Street 1', str.upper).distinct('Cross Street 1')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(cross_street_1)

In [ ]:
cross_street_1_dict = {}
cluster_dict = {}
def print_k_clusters_and_integrate_cs1(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')
          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        cross_street_1_dict[i] = temp_dict

print_k_clusters_and_integrate_cs1(clusters)

In [ ]:
final_cs1 = {}
for key,value in cross_street_1_dict.items():
    final_cs1[key] = max(value, key=value.get)

In [ ]:
@udf

def update_cross_street_1(street_name):
    if street_name in cluster_dict:
      return final_cs1[cluster_dict[street_name]]
    return " "
df_311= df_311.withColumnRenamed("Cross Street 1","Cross_Street_1")
df_311= df_311.withColumn("Cross_Street_1", update_cross_street_1(df_311.Cross_Street_1))
df_311= df_311.withColumn("Cross_Street_1", replace_with_single_spaces(df_311.Cross_Street_1))

#### Cross Street 2

In [ ]:
cross_street_2 = ds.update('Cross Street 2', str.upper).distinct('Cross Street 2')
clusters = KeyCollision(func=Fingerprint(), threads=3).clusters(cross_street_2)

In [ ]:
cross_street_2_dict = {}
cluster_dict = {}
def print_k_clusters_and_integrate_cs2(clusters):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(len(clusters)):
        print('\nCluster {}'.format(i + 1))
        temp_dict = {}
        for key, cnt in clusters[i].items():
          if key == '':
            key = "''"
          print(f'  {key} (x {cnt})')

          temp_dict[key] = int(cnt)
          cluster_dict[key] = i
        cross_street_2_dict[i] = temp_dict

print_k_clusters_and_integrate_cs2(clusters)

In [ ]:
final_cs2 = {}
for key,value in cross_street_2_dict.items():
    final_cs2[key] = max(value, key=value.get)

In [ ]:
@udf

def update_cross_street_2(street_name):
    if street_name in cluster_dict:
      return final_cs2[cluster_dict[street_name]]
    return " "
df_311= df_311.withColumnRenamed("Cross Street 2","Cross_Street_2")
df_311= df_311.withColumn("Cross_Street_2", update_cross_street_2(df_311.Cross_Street_2))
df_311= df_311.withColumn("Cross_Street_1", replace_with_single_spaces(df_311.Cross_Street_1))

### Cleaning Longitude and Latitude

In [ ]:
df_311.na.drop(subset=["Longitude","Latitude"]).show()

In [ ]:
df_311.select("Longitude").distinct().show(50)

In [ ]:
df_311.select("Latitude").distinct().show(50)

Checking for outliers, we find no outliers 

In [ ]:
df_311[['Latitude','Longitude']].summary().show()

We Find no Function Dependancy Violations between (latittude,Longitude) -> Borough indicating the data is bynow already it's in purest form.

In [ ]:
df100k = df_311.limit(10000)
df = df100k.toPandas()
boro_violations = fd_violations(df, ['Longitude', 'Latitude'],['Borough'])
print('# of violations for Longitude, Latitude -> Borough is {}\n'.format(len(boro_violations)))
for key, gr in boro_violations.items():
  print(gr[['Longitude', 'Latitude','Borough']])

###Save to a file

In [ ]:
df_311.write.format("csv").save("cleaned_data")

In [ ]:
df_311.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("cleaned_data")


In [ ]:
df_311.printSchema()


root
 |-- Unique Key: string (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Resolution Description: string (nullable = true)
 |-- Resolution Action Updated Date: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- BBL: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- X Coordinate (State Plane): string (nullable = true)
 |-- Y Coord